In [503]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
from datetime import datetime
pd.set_option('display.max_rows', None)

In [544]:
path = '/Users/USER/Dropbox/台北最愛排房表/最愛109年訂房房況表.xls'

In [545]:
xls = pd.ExcelFile(path)

In [546]:
# read in the slide of excel
df = pd.read_excel(xls,'109年7-12月')

# Data Cleannig

In [547]:
df = df.iloc[:18,1:].drop(1).T

In [548]:
column_name = ['days_in_week',201,
 301,
 501,
 601,
 202,
 302,
 502,
 602,
 203,
 303,
 503,
 603,
 205,
 305,
 505,
 605]

In [549]:
#get relevant column
df.columns = column_name

In [550]:
# get relevant rows
df = df.iloc[1:]

# Data Manupilation

In [551]:
#get the boolean of current relevant day
current_date_boolean = pd.Series(df.index) >= date.today()

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  


In [552]:
# resettig index to allow date to be accessed
df = df.reset_index()
df.rename(columns={'index':'date'}, inplace=True)

In [553]:
#filtering the date
df = df[current_date_boolean]

In [554]:
df.head()

,date,days_in_week,201,301,501,601,202,302,502,602,203,303,503,603,205,305,505,605
115,2020-10-24,六,許文益(防疫/$2500/含餐),HOANG越南移工($2300含餐),林仲甫($2500/含餐),張嘉德($2500/含餐),NaN,吳佳容($2500/含餐),陳盈元($2500/含餐),代愛琴(防疫/$2500/含餐),NaN,林永欣((防疫/$2500/含餐),陳雅明($2500/含餐),翁藝芳(防疫/$2500/含餐),NaN,張順傑($2600/含餐),顏瑞華(防疫/$2800/含餐)付清,沈韋彤($2800/含餐)
116,2020-10-25,日,許文益(防疫/$2500/含餐),HOANG越南移工($2300含餐),林仲甫($2500/含餐),張嘉德($2500/含餐),NaN,吳佳容($2500/含餐),陳盈元($2500/含餐),代愛琴(防疫/$2500/含餐),NaN,林永欣((防疫/$2500/含餐),陳雅明($2500/含餐),翁藝芳(防疫/$2500/含餐),NaN,張順傑($2600/含餐),NaN,沈韋彤($2800/含餐)
117,2020-10-26,一,NaN,HOANG越南移工($2300含餐),林仲甫($2500/含餐),張嘉德($2500/含餐),NaN,吳佳容($2500/含餐),陳盈元($2500/含餐),NaN,NaN,NaN,陳雅明($2500/含餐),NaN,NaN,張順傑($2600/含餐),NaN,沈韋彤($2800/含餐)
118,2020-10-27,二,NaN,HOANG越南移工($2300含餐),林仲甫($2500/含餐),張嘉德($2500/含餐),NaN,吳佳容($2500/含餐),陳盈元($2500/含餐),NaN,NaN,NaN,陳雅明($2500/含餐),LE THI越外配10/27~11/11($2500含餐)-10.21有匯2500訂金,NaN,張順傑($2600/含餐),NaN,沈韋彤($2800/含餐)
119,2020-10-28,三,莊小姐10/28~11/11($2000不含餐),HOANG越南移工($2300含餐),林仲甫($2500/含餐),張嘉德($2500/含餐),NaN,吳佳容($2500/含餐),陳盈元($2500/含餐),NaN,NaN,NaN,陳雅明($2500/含餐),LE THI越外配($2500含餐),NaN,張順傑($2600/含餐),NaN,沈韋彤($2800/含餐)


In [555]:
#get rooms
rooms = [201,
 301,
 501,
 601,
 202,
 302,
 502,
 602,
 203,
 303,
 503,
 603,
 205,
 305,
 505,
 605]

# Stratify the dates that allow reservation

In [556]:
final_room_ava = pd.DataFrame()

In [557]:
#pre-allocation of number of days possible
ava_series_econ = pd.Series([0]*df.shape[0])
ava_series_stand = pd.Series([0]*df.shape[0])
ava_series_deluxe = pd.Series([0]*df.shape[0])

In [558]:
#stratify each room for 15 days continuous vacancies
for j in range(len(rooms)):
    temp_name = df.iloc[:,j+2].name
    temp_room_vacancy = df[temp_name].isna()
    for i in range(df.shape[0]-14):
        if str(temp_name)[1:] =='01':
            if temp_room_vacancy.iloc[i:i+15].sum() == 15:
                ava_series_econ[i] += 1
        elif str(temp_name)[1:] =='05':
            if temp_room_vacancy.iloc[i:i+15].sum() == 15:
                ava_series_deluxe[i] += 1
        else:
            if temp_room_vacancy.iloc[i:i+15].sum() == 15:
                ava_series_stand[i] += 1

In [559]:
final_room_ava['date'] = list(df['date'])
final_room_ava['經濟'] = ava_series_econ
final_room_ava['標準'] = ava_series_stand
final_room_ava['豪華'] = ava_series_deluxe

In [560]:
export = (final_room_ava.set_index('date').T)

In [561]:
export

date,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,...,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31
經濟,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
標準,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
豪華,1,2,2,2,2,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0


In [562]:
#changing it to format friendly to excel
export.columns = pd.Series(export.columns).apply(lambda x: str(x.date().strftime('%m-%d')))

In [563]:
export.to_excel(r'/Users/USER/Dropbox/台北最愛排房表/今天空房表.xls')

In [564]:
export

date,10-24,10-25,10-26,10-27,10-28,10-29,10-30,10-31,11-01,11-02,...,12-22,12-23,12-24,12-25,12-26,12-27,12-28,12-29,12-30,12-31
經濟,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
標準,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
豪華,1,2,2,2,2,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
